In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/inputdata/studentRegistration.csv
/kaggle/input/inputdata/studentAssessment.csv
/kaggle/input/inputdata/studentInfo.csv
/kaggle/input/inputdata/studentVle.csv
/kaggle/input/inputdata/vle.csv
/kaggle/input/inputdata/Test_Who.csv
/kaggle/input/inputdata/Submission_reg.csv
/kaggle/input/inputdata/Train_Who.csv
/kaggle/input/inputdata/assessments.csv
/kaggle/input/inputdata/courses.csv
/kaggle/input/inputdata/Train_Target_reg.csv
/kaggle/input/urfu-sf-2024/__results__.html
/kaggle/input/urfu-sf-2024/__notebook__.ipynb
/kaggle/input/urfu-sf-2024/__output__.json
/kaggle/input/urfu-sf-2024/custom.css
/kaggle/input/urfu-sf-2024/__results___files/__results___36_0.png


In [48]:
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
test_who = pd.read_csv('/kaggle/input/inputdata/Test_Who.csv')
train_who = pd.read_csv('/kaggle/input/inputdata/Train_Who.csv')
train_target_reg = pd.read_csv('/kaggle/input/inputdata/Train_Target_reg.csv', index_col = 'ID')
submission = pd.read_csv('/kaggle/input/inputdata/Submission_reg.csv', index_col = 'ID')

# Данные
assessments = pd.read_csv('/kaggle/input/inputdata/assessments.csv') 

stud_ass = pd.read_csv('/kaggle/input/inputdata/studentAssessment.csv')
stud_ass['id_student'] = stud_ass['id_student'].astype(object)

student_info = pd.read_csv('/kaggle/input/inputdata/studentInfo.csv')
student_info['id_student'] = student_info['id_student'].astype(object) 

student_reg = pd.read_csv('/kaggle/input/inputdata/studentRegistration.csv')
student_reg['id_student'] = student_reg['id_student'].astype(object) 

student_vle = pd.read_csv('/kaggle/input/inputdata/studentVle.csv')
student_vle['id_student'] = student_vle['id_student'].astype(object) 
student_vle['id_site'] = student_vle['id_site'].astype(object) 
Test_Who = pd.read_csv('/kaggle/input/inputdata/Test_Who.csv')

In [50]:
# Submission_reg - Пример файла для загрузки, в который проставлены все средние значения
# Test_Who - перечень id_student, code_modelu и code_presentation для тестовой выборки. ID - внутренний идентификатор для оценки на Kaggle
# Train_Target_reg - целевая метрика для тренировочной выборки. ID - внутренний идентификатор для оценки на Kaggle, как в файле Train_Who.csv
# Train_Who - перечень id_student, code_modelu и code_presentation для тренировочной выборки. ID - внутренний идентификатор для оценки на Kaggle
# assessments - Файл содержит информацию об оценках в модулях-презентациях. Обычно каждая презентация имеет ряд оценок, за которыми следует итоговый экзамен.
# courses -  Файл содержит список всех доступных модулей и их презентаций.
# studentAssessment - Файл содержит результаты оценок студентов. Если учащийся не сдает оценку, результат не записывается.
# studentInfo -  Файл содержит демографическую информацию об учащихся
# studentRegistration - Файл содержит информацию о времени регистрации студента на презентацию модуля.
# studentVle - Фйал содержит информацию о взаимодействии каждого учащегося с материалами в VLE
# vle - Файл содержит информацию о доступных материалах в VLE (Virtual Learning Environment). Обычно это html-страницы, pdf-файлы и т. д.

In [106]:
Test_Who = pd.read_csv('/kaggle/input/inputdata/Test_Who.csv')

In [107]:
Test_Who

,ID,id_student,code_module,code_presentation
0,26074,2588655,DDD,2014J
1,26075,543628,CCC,2014B
2,26076,387276,BBB,2013B
3,26077,692219,CCC,2014J
4,26078,516212,BBB,2013J
...,...,...,...,...
6514,32588,230510,DDD,2013J
6515,32589,384026,DDD,2013B
6516,32590,2367155,AAA,2013J
6517,32591,501444,FFF,2014J


In [51]:
agged_stud_ass = stud_ass.groupby('id_student').agg(mean_date_submitted=('date_submitted', 'mean'), sum=('is_banked', 'sum'))
del stud_ass
agged_stud_ass

,mean_date_submitted,sum
id_student,,
6516,111.600000,0
8462,23.000000,4
11391,112.400000,0
23629,55.750000,0
23698,133.444444,0
...,...,...
2698251,121.428571,0
2698257,116.200000,0
2698535,90.375000,0


In [52]:
agged_student_vle = student_vle.groupby(['id_student']).agg(sum_click=('sum_click', 'sum')).reset_index()
del student_vle # рама лишней не бывает, очищаем память
agged_student_vle

,id_student,sum_click
0,6516,2791
1,8462,656
2,11391,934
3,23629,161
4,23698,910
...,...,...
26069,2698251,1511
26070,2698257,758
26071,2698535,4241
26072,2698577,717


In [53]:
agged_vle_ass = agged_student_vle.join(agged_stud_ass, on=['id_student'], how='inner')
del agged_student_vle
agged_vle_ass

,id_student,sum_click,mean_date_submitted,sum
0,6516,2791,111.600000,0
1,8462,656,23.000000,4
2,11391,934,112.400000,0
3,23629,161,55.750000,0
4,23698,910,133.444444,0
...,...,...,...,...
26069,2698251,1511,121.428571,0
26070,2698257,758,116.200000,0
26071,2698535,4241,90.375000,0
26072,2698577,717,106.200000,0


In [54]:
info_reg = (
    pd.merge(
        student_reg, 
        student_info, 
        how='inner', 
        on=['id_student', 'code_module', 'code_presentation'])
)

del student_reg
del student_info
info_reg

,code_module,code_presentation,id_student,date_registration,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability
0,AAA,2013J,11391,-159.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N
1,AAA,2013J,28400,-53.0,F,Scotland,HE Qualification,20-30%,35-55,0,60,N
2,AAA,2013J,30268,-92.0,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y
3,AAA,2013J,31604,-52.0,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N
4,AAA,2013J,32885,-176.0,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N
...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,-4.0,F,Wales,Lower Than A Level,10-20,0-35,0,30,N
32589,GGG,2014J,2645731,-23.0,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N
32590,GGG,2014J,2648187,-129.0,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y
32591,GGG,2014J,2679821,-49.0,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N


In [55]:
info_reg_vle_ass = (
    pd.merge(
        info_reg, 
        agged_vle_ass, 
        how='inner', 
        on=['id_student']
    )
)

del info_reg
del agged_vle_ass
info_reg_vle_ass

,code_module,code_presentation,id_student,date_registration,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,mean_date_submitted,sum
0,AAA,2013J,11391,-159.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,934,112.400000,0
1,AAA,2013J,28400,-53.0,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,1435,114.200000,0
2,AAA,2013J,31604,-52.0,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,2158,112.200000,0
3,AAA,2013J,32885,-176.0,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,1034,125.600000,0
4,AAA,2013J,38053,-110.0,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,2445,116.200000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26716,GGG,2014J,2620947,-23.0,F,Scotland,A Level or Equivalent,80-90%,0-35,0,30,Y,476,161.000000,0
26717,GGG,2014J,2645731,-23.0,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,893,156.222222,0
26718,GGG,2014J,2648187,-129.0,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,312,147.111111,0
26719,GGG,2014J,2679821,-49.0,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,275,65.500000,0


In [56]:
agg_ass = (
    assessments
        .groupby(['code_module', 'code_presentation'])
        .agg(
            mean_date=('date', 'mean'), 
            mean_weight=('weight', 'mean')
        )
        .reset_index()
)
agg_ass

,code_module,code_presentation,mean_date,mean_weight
0,AAA,2013J,114.200000,33.333333
1,AAA,2014J,114.200000,33.333333
2,BBB,2013B,112.545455,16.666667
3,BBB,2013J,120.181818,16.666667
4,BBB,2014B,108.090909,16.666667
5,BBB,2014J,107.200000,33.333333
6,CCC,2014B,114.250000,30.000000
7,CCC,2014J,118.625000,30.000000
8,DDD,2013B,118.428571,14.285714
9,DDD,2013J,131.714286,28.571429


In [57]:
info_reg_vle_ass_ass = (
    pd.merge(
        info_reg_vle_ass, 
        agg_ass,
        how='inner', 
        on=['code_module', 'code_presentation']
    )
)
del info_reg_vle_ass
info_reg_vle_ass_ass
info_reg_vle_ass_ass

,code_module,code_presentation,id_student,date_registration,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,mean_date_submitted,sum,mean_date,mean_weight
0,AAA,2013J,11391,-159.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,934,112.400000,0,114.2,33.333333
1,AAA,2013J,28400,-53.0,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,1435,114.200000,0,114.2,33.333333
2,AAA,2013J,31604,-52.0,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,2158,112.200000,0,114.2,33.333333
3,AAA,2013J,32885,-176.0,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,1034,125.600000,0,114.2,33.333333
4,AAA,2013J,38053,-110.0,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,2445,116.200000,0,114.2,33.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26716,GGG,2014J,2620947,-23.0,F,Scotland,A Level or Equivalent,80-90%,0-35,0,30,Y,476,161.000000,0,196.1,10.000000
26717,GGG,2014J,2645731,-23.0,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,893,156.222222,0,196.1,10.000000
26718,GGG,2014J,2648187,-129.0,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,312,147.111111,0,196.1,10.000000
26719,GGG,2014J,2679821,-49.0,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,275,65.500000,0,196.1,10.000000


In [58]:
train = (
    pd.merge(
        train_who, 
        info_reg_vle_ass_ass, 
        how='left', 
        on=['id_student', 'code_module', 'code_presentation']
    )
)

del train_who
train

,ID,id_student,code_module,code_presentation,date_registration,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,mean_date_submitted,sum,mean_date,mean_weight
0,0,464685,DDD,2014B,-36.0,F,East Anglian Region,A Level or Equivalent,50-60%,0-35,1.0,70.0,N,1694.0,123.285714,0.0,123.857143,28.571429
1,1,606593,DDD,2013J,-32.0,M,Scotland,A Level or Equivalent,20-30%,0-35,0.0,120.0,N,692.0,55.333333,0.0,131.714286,28.571429
2,2,1674799,CCC,2014J,-21.0,M,South Region,HE Qualification,70-80%,55<=,0.0,60.0,N,4819.0,135.500000,0.0,118.625000,30.000000
3,3,195246,FFF,2014J,-124.0,M,West Midlands Region,Lower Than A Level,10-20,0-35,0.0,60.0,N,2138.0,107.181818,0.0,187.153846,15.384615
4,4,582080,FFF,2013J,-114.0,M,South East Region,Lower Than A Level,10-20,35-55,0.0,60.0,N,188.0,19.000000,0.0,181.076923,15.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26069,26069,588477,AAA,2014J,-141.0,M,Yorkshire Region,HE Qualification,20-30%,0-35,0.0,60.0,N,5183.0,112.800000,0.0,114.200000,33.333333
26070,26070,653826,BBB,2014J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26071,26071,126411,BBB,2013B,-124.0,F,West Midlands Region,Lower Than A Level,0-10%,0-35,2.0,60.0,Y,25.0,60.000000,0.0,112.545455,16.666667
26072,26072,547448,DDD,2013J,-169.0,M,South Region,Lower Than A Level,90-100%,35-55,0.0,60.0,N,1740.0,123.714286,0.0,131.714286,28.571429


In [59]:
test = (
    pd.merge(
        test_who, 
        info_reg_vle_ass_ass, 
        how='left', 
        on=['id_student', 'code_module', 'code_presentation'],
    )
)

del test_who
del info_reg_vle_ass_ass
test

,ID,id_student,code_module,code_presentation,date_registration,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,mean_date_submitted,sum,mean_date,mean_weight
0,26074,2588655,DDD,2014J,-92.0,M,Scotland,Lower Than A Level,50-60%,0-35,2.0,60.0,N,414.0,85.777778,3.0,95.833333,28.571429
1,26075,543628,CCC,2014B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26076,387276,BBB,2013B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26077,692219,CCC,2014J,-34.0,M,South Region,A Level or Equivalent,20-30%,0-35,0.0,90.0,N,549.0,21.000000,0.0,118.625000,30.000000
4,26078,516212,BBB,2013J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6514,32588,230510,DDD,2013J,-54.0,M,South Region,Lower Than A Level,10-20,0-35,0.0,60.0,Y,167.0,41.500000,0.0,131.714286,28.571429
6515,32589,384026,DDD,2013B,-34.0,F,Yorkshire Region,A Level or Equivalent,30-40%,0-35,1.0,120.0,N,168.0,62.666667,0.0,118.428571,14.285714
6516,32590,2367155,AAA,2013J,-22.0,F,East Anglian Region,A Level or Equivalent,80-90%,0-35,0.0,120.0,N,2148.0,114.200000,0.0,114.200000,33.333333
6517,32591,501444,FFF,2014J,-59.0,M,South Region,Lower Than A Level,10-20,0-35,0.0,60.0,N,3555.0,131.916667,0.0,187.153846,15.384615


In [60]:
train.index = train.ID
train.drop(['ID'], axis=1, inplace=True)
test.index = test.ID
test.drop(['ID'], axis=1, inplace=True)

In [61]:
train

,id_student,code_module,code_presentation,date_registration,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,mean_date_submitted,sum,mean_date,mean_weight
ID,,,,,,,,,,,,,,,,,
0,464685,DDD,2014B,-36.0,F,East Anglian Region,A Level or Equivalent,50-60%,0-35,1.0,70.0,N,1694.0,123.285714,0.0,123.857143,28.571429
1,606593,DDD,2013J,-32.0,M,Scotland,A Level or Equivalent,20-30%,0-35,0.0,120.0,N,692.0,55.333333,0.0,131.714286,28.571429
2,1674799,CCC,2014J,-21.0,M,South Region,HE Qualification,70-80%,55<=,0.0,60.0,N,4819.0,135.500000,0.0,118.625000,30.000000
3,195246,FFF,2014J,-124.0,M,West Midlands Region,Lower Than A Level,10-20,0-35,0.0,60.0,N,2138.0,107.181818,0.0,187.153846,15.384615
4,582080,FFF,2013J,-114.0,M,South East Region,Lower Than A Level,10-20,35-55,0.0,60.0,N,188.0,19.000000,0.0,181.076923,15.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26069,588477,AAA,2014J,-141.0,M,Yorkshire Region,HE Qualification,20-30%,0-35,0.0,60.0,N,5183.0,112.800000,0.0,114.200000,33.333333
26070,653826,BBB,2014J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26071,126411,BBB,2013B,-124.0,F,West Midlands Region,Lower Than A Level,0-10%,0-35,2.0,60.0,Y,25.0,60.000000,0.0,112.545455,16.666667


In [62]:
test.isna().sum()

id_student                 0
code_module                0
code_presentation          0
date_registration       1140
gender                  1138
region                  1138
highest_education       1138
imd_band                1337
age_band                1138
num_of_prev_attempts    1138
studied_credits         1138
disability              1138
sum_click               1138
mean_date_submitted     1138
sum                     1138
mean_date               1138
mean_weight             1138
dtype: int64

In [63]:
cat_columns = [] # создаем пустой список для имен колонок категориальных данных
num_columns = [] # создаем пустой список для имен колонок числовых данных

for column_name in train.columns: # смотрим на все колонки в датафрейме
    if (train[column_name].dtypes == object): # проверяем тип данных для каждой колонки
        cat_columns +=[column_name] # если тип объект - то складываем в категориальные данные
    else:
        num_columns +=[column_name] # иначе - числовые

# важно: если признак категориальный, но хранится в формате числовых данных, тогда код не сработает корректно


# выводим результат
print('Категориальные данные:\t ',cat_columns, '\n Число столблцов = ',len(cat_columns))

print('Числовые данные:\t ',  num_columns, '\n Число столблцов = ',len(num_columns))

Категориальные данные:	  ['id_student', 'code_module', 'code_presentation', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability'] 
 Число столблцов =  9
Числовые данные:	  ['date_registration', 'num_of_prev_attempts', 'studied_credits', 'sum_click', 'mean_date_submitted', 'sum', 'mean_date', 'mean_weight'] 
 Число столблцов =  8


In [64]:
train_target_reg

,weighted_score
ID,
0,64.9125
1,14.8625
2,61.4300
3,39.5000
4,9.2500
...,...
26069,75.0000
26070,75.0000
26071,1.0000


In [65]:
train_df = pd.concat([train, train_target_reg], axis=1)

Обработка колонки code_module

In [66]:
# Создаем дамми-переменные
dummies = pd.get_dummies(train_df['code_module'], prefix='code_module', drop_first=True, dtype=int)
# Объединяем исходный датафрейм с дамми-переменными
train_df = pd.concat([train_df, dummies], axis=1)
# Удаляем исходную колонку
train_df.drop('code_module', axis=1, inplace=True)

###########################
# Тоже самое для test
dummies = pd.get_dummies(test['code_module'], prefix='code_module', drop_first=True, dtype=int)
# Объединяем исходный датафрейм с дамми-переменными
test = pd.concat([test, dummies], axis=1)
# Удаляем исходную колонку
test.drop('code_module', axis=1, inplace=True)


Обработка колонки code_presentation

In [67]:
# Создаем дамми-переменные
dummies = pd.get_dummies(train_df['code_presentation'], prefix='code_presentation', drop_first=True, dtype=int)

# Объединяем исходный датафрейм с дамми-переменными
train_df = pd.concat([train_df, dummies], axis=1)

# Удаляем исходную колонку
train_df.drop('code_presentation', axis=1, inplace=True)

########################################################
# Тоже самое для test
dummies = pd.get_dummies(test['code_presentation'], prefix='code_presentation', drop_first=True, dtype=int)

# Объединяем исходный датафрейм с дамми-переменными
test = pd.concat([test, dummies], axis=1)

# Удаляем исходную колонку
test.drop('code_presentation', axis=1, inplace=True)

Обработка колонки region 

In [68]:
# Заменяем пропущенные значения на строку 'Unknown'
train_df['region'].fillna('Unknown', inplace=True)

# Создаем дамми-переменные
dummies = pd.get_dummies(train_df['region'], prefix='region', drop_first=True, dtype=int)

# Объединяем исходный датафрейм с дамми-переменными
train_df = pd.concat([train_df, dummies], axis=1)

# Удаляем исходную колонку 'region'
train_df.drop('region', axis=1, inplace=True)

#####################
# Тоже самое для test
test['region'].fillna('Unknown', inplace=True)

# Создаем дамми-переменные
dummies = pd.get_dummies(test['region'], prefix='region', drop_first=True, dtype=int)

# Объединяем исходный датафрейм с дамми-переменными
test = pd.concat([test, dummies], axis=1)

# Удаляем исходную колонку 'region'
test.drop('region', axis=1, inplace=True)


/tmp/ipykernel_32/1465918558.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['region'].fillna('Unknown', inplace=True)
/tmp/ipykernel_32/1465918558.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

Обработка колонки highest_education 

In [69]:
# Заменяем пропущенные значения на строку 'Unknown'
train_df['highest_education'].fillna('Unknown_education', inplace=True)

# Создаем дамми-переменные
dummies = pd.get_dummies(train_df['highest_education'], prefix='highest_education', drop_first=True, dtype=int)

# Объединяем исходный датафрейм с дамми-переменными
train_df = pd.concat([train_df, dummies], axis=1)

# Удаляем исходную колонку
train_df.drop('highest_education', axis=1, inplace=True)

######################
# Тоже самое для test
test['highest_education'].fillna('Unknown_education', inplace=True)

# Создаем дамми-переменные
dummies = pd.get_dummies(test['highest_education'], prefix='highest_education', drop_first=True, dtype=int)

# Объединяем исходный датафрейм с дамми-переменными
test = pd.concat([test, dummies], axis=1)

# Удаляем исходную колонку
test.drop('highest_education', axis=1, inplace=True)

/tmp/ipykernel_32/1693164047.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['highest_education'].fillna('Unknown_education', inplace=True)
/tmp/ipykernel_32/1693164047.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [71]:
train_df.loc[train_df['imd_band'] == '50-60%', 'imd_band'] = 55
train_df.loc[train_df['imd_band'] == '20-30%', 'imd_band'] = 25
train_df.loc[train_df['imd_band'] == '70-80%', 'imd_band'] = 75
train_df.loc[train_df['imd_band'] == '10-20', 'imd_band'] = 15
train_df.loc[train_df['imd_band'] == '40-50%', 'imd_band'] = 45
train_df.loc[train_df['imd_band'] == '30-40%', 'imd_band'] = 35
train_df.loc[train_df['imd_band'] == '90-100%', 'imd_band'] = 95
train_df.loc[train_df['imd_band'] == '60-70%', 'imd_band'] = 65
train_df.loc[train_df['imd_band'] == '80-90%', 'imd_band'] = 85
train_df.loc[train_df['imd_band'] == '0-10%', 'imd_band'] = 5
train_df.loc[train_df['imd_band'].isna(), 'imd_band'] = round(train_df['imd_band'].mean(), 0)
train_df['imd_band'] = train_df['imd_band'].astype('int16')

###########################
# Тоже самое для test
test.loc[test['imd_band'] == '50-60%', 'imd_band'] = 55
test.loc[test['imd_band'] == '20-30%', 'imd_band'] = 25
test.loc[test['imd_band'] == '70-80%', 'imd_band'] = 75
test.loc[test['imd_band'] == '10-20', 'imd_band'] = 15
test.loc[test['imd_band'] == '40-50%', 'imd_band'] = 45
test.loc[test['imd_band'] == '30-40%', 'imd_band'] = 35
test.loc[test['imd_band'] == '90-100%', 'imd_band'] = 95
test.loc[test['imd_band'] == '60-70%', 'imd_band'] = 65
test.loc[test['imd_band'] == '80-90%', 'imd_band'] = 85
test.loc[test['imd_band'] == '0-10%', 'imd_band'] = 5
test.loc[test['imd_band'].isna(), 'imd_band'] = round(test['imd_band'].mean(), 0)
test['imd_band'] = test['imd_band'].astype('int16')

Обработка колонки age_band 

In [72]:
# Заменяем пропущенные значения на строку 'Unknown'
train_df['age_band'].fillna('Unknown_age_band', inplace=True)

# Создаем дамми-переменные
dummies = pd.get_dummies(train_df['age_band'], prefix='age_band', drop_first=True, dtype=int)

# Объединяем исходный датафрейм с дамми-переменными
train_df = pd.concat([train_df, dummies], axis=1)

# Удаляем исходную колонку
train_df.drop('age_band', axis=1, inplace=True)

#######################
# Тоже самое для test
test['age_band'].fillna('Unknown_age_band', inplace=True)

# Создаем дамми-переменные
dummies = pd.get_dummies(test['age_band'], prefix='age_band', drop_first=True, dtype=int)

# Объединяем исходный датафрейм с дамми-переменными
test = pd.concat([test, dummies], axis=1)

# Удаляем исходную колонку
test.drop('age_band', axis=1, inplace=True)

/tmp/ipykernel_32/4221552464.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['age_band'].fillna('Unknown_age_band', inplace=True)
/tmp/ipykernel_32/4221552464.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

In [73]:
train_df['disability'].unique()

array(['N', nan, 'Y'], dtype=object)

In [74]:
# # Задаем отношение частоты значений 'N' к 'Y'
# # N встречается в 9.54 раза чаще чем Y
# ratio_N_to_Y = 9.54

# # Подсчитываем количество значений 'N', 'Y' и пропущенных значений в колонке 'disability'
# n_N = train_df['disability'].str.count('N').sum()
# n_Y = train_df['disability'].str.count('Y').sum()
# n_missing = train_df['disability'].isnull().sum()

# # Вычисляем вероятности P(N) и P(Y)
# P_N = n_N / (n_N + n_Y)
# P_Y = 1 - P_N

# # Вычисляем вероятность P(N) в 9.54 раза больше, чем P(Y)
# P_N_new = 9.54 * P_Y
# P_Y_new = 1 - P_N_new

# # Распределяем пропущенные значения в соответствии с вероятностями P(N) и P(Y)
# missing_indices = train_df[train_df['disability'].isnull()].index
# replace_N = int(np.round(P_N_new / (P_N_new + P_Y_new) * n_missing))
# replace_Y = n_missing - replace_N

# # Заполняем пропущенные значения 'N' и 'Y' в колонке 'disability'
# train_df.loc[np.random.choice(missing_indices, replace_N, replace=False), 'disability'] = 'N'
# train_df.loc[np.random.choice(missing_indices, replace_Y, replace=False), 'disability'] = 'Y'

In [ ]:
# ###################
# # Тоже самое для test
# # Задаем отношение частоты значений 'N' к 'Y'
# # N встречается в 9.54 раза чаще чем Y
# ratio_N_to_Y = 9.54

# # Подсчитываем количество значений 'N', 'Y' и пропущенных значений в колонке 'disability'
# n_N = test['disability'].str.count('N').sum()
# n_Y = test['disability'].str.count('Y').sum()
# n_missing = test['disability'].isnull().sum()

# # Вычисляем вероятности P(N) и P(Y)
# P_N = n_N / (n_N + n_Y)
# P_Y = 1 - P_N

# # Вычисляем вероятность P(N) в 9.54 раза больше, чем P(Y)
# P_N_new = 9.54 * P_Y
# P_Y_new = 1 - P_N_new

# # Распределяем пропущенные значения в соответствии с вероятностями P(N) и P(Y)
# missing_indices = test[test['disability'].isnull()].index
# replace_N = int(np.round(P_N_new / (P_N_new + P_Y_new) * n_missing))
# replace_Y = n_missing - replace_N

# # Заполняем пропущенные значения 'N' и 'Y' в колонке 'disability'
# test.loc[np.random.choice(missing_indices, replace_N, replace=False), 'disability'] = 'N'
# test.loc[np.random.choice(missing_indices, replace_Y, replace=False), 'disability'] = 'Y'

In [75]:
# date_registration непонятно что за колонка, почему какие то числа отрицательные, какие то положительныеtrain_df
train_df = train_df.drop(['date_registration', 'id_student'], axis=1)
test = test.drop(['date_registration', 'id_student'], axis=1)

In [76]:
train_df.loc[train_df['gender'] == 'M', 'gender'] = 1
train_df.loc[train_df['gender'] == 'F', 'gender'] = 0
gender_array = np.random.randint(0, 2, len(train_df[train_df['gender'].isna()]))
train_df.loc[train_df['gender'].isna(), 'gender'] = gender_array
train_df['gender'] = train_df['gender'].astype('int8')

###############
# test
test.loc[test['gender'] == 'M', 'gender'] = 1
test.loc[test['gender'] == 'F', 'gender'] = 0
gender_array = np.random.randint(0, 2, len(test[test['gender'].isna()]))
test.loc[test['gender'].isna(), 'gender'] = gender_array
test['gender'] = test['gender'].astype('int8')

In [ ]:
# train_df.loc[train_df['disability'] == 'Y', 'gender'] = 1
# train_df.loc[train_df['disability'] == 'N', 'gender'] = 0

# # Студенты, которые имеют инфалидность примерно в 10 раз меньше тех, кто не имеет инвалидность
# # Выбираем индексы строк с пропущенными значениями в колонке 'disability'
# missing_indices = train_df[train_df['disability'].isnull()].index

# # Генерируем случайные индексы для заполнения половины пропущенных значений
# random_indices = np.random.choice(missing_indices, len(missing_indices)//10, replace=False)

# # Заполняем десятую часть пропущенных значений = 1
# train_df.loc[random_indices, 'disability'] = 1

# # Остальные данные заполняем 0
# train_df.loc[train_df['disability'].isna(), 'disability'] = 0

# #######################
# # Тоже самое для test
# test.loc[test['disability'] == 'Y', 'gender'] = 1
# test.loc[test['disability'] == 'N', 'gender'] = 0

# # Студенты, которые имеют инфалидность примерно в 10 раз меньше тех, кто не имеет инвалидность
# # Выбираем индексы строк с пропущенными значениями в колонке 'disability'
# missing_indices = test[test['disability'].isnull()].index

# # Генерируем случайные индексы для заполнения половины пропущенных значений
# random_indices = np.random.choice(missing_indices, len(missing_indices)//10, replace=False)

# # Заполняем десятую часть пропущенных значений = 1
# test.loc[random_indices, 'disability'] = 1

# # Остальные данные заполняем 0
# test.loc[test['disability'].isna(), 'disability'] = 0


In [77]:
train_df.loc[train_df['disability'] == 'N', 'disability'] = 0
train_df.loc[train_df['disability'] == 'Y', 'disability'] = 1
test.loc[test['disability'] == 'N', 'disability'] = 0
test.loc[test['disability'] == 'Y', 'disability'] = 1

# ЗАПОЛНЕНИЕ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ

In [78]:
from sklearn.impute import SimpleImputer

# для train_df
imputer = SimpleImputer(strategy='most_frequent')  # 'most_frequent' используется для заполнения пропущенных значений самым частым значением

columns_to_impute = ['disability', 'num_of_prev_attempts', 'studied_credits', 'sum_click', 'mean_date_submitted', 'sum', 'mean_date', 'mean_weight']

train_df[columns_to_impute] = imputer.fit_transform(train_df[columns_to_impute])

# для test
imputer = SimpleImputer(strategy='most_frequent')  # 'most_frequent' используется для заполнения пропущенных значений самым частым значением

columns_to_impute = ['disability', 'num_of_prev_attempts', 'studied_credits', 'sum_click', 'mean_date_submitted', 'sum', 'mean_date', 'mean_weight']

test[columns_to_impute] = imputer.fit_transform(test[columns_to_impute])


# Преобразование типов данных

In [ ]:
test

In [79]:
# Для train_df
train_df['num_of_prev_attempts'] = train_df['num_of_prev_attempts'].astype('int8')
train_df['imd_band'] = train_df['imd_band'].astype('int16')
train_df['num_of_prev_attempts'] = train_df['num_of_prev_attempts'].astype('int8')
train_df['studied_credits'] = train_df['studied_credits'].astype('int16')
train_df['disability'] = train_df['disability'].astype('int8')
train_df['sum_click'] = train_df['sum_click'].astype('int32')
train_df['sum'] = train_df['sum'].astype('int8')
train_df['mean_date_submitted'] = train_df['mean_date_submitted'].astype('float64')
train_df['mean_weight'] = train_df['mean_weight'].astype('float64')
train_df['mean_date'] = train_df['mean_date'].astype('float64')

cols_to_convert = train_df.columns[10:]  # Выбираем столбцы с 10-го по последний
# Преобразуем типы данных столбцов с int64 на int8
train_df[cols_to_convert] = train_df[cols_to_convert].astype('int8')


# Для test
test['num_of_prev_attempts'] = test['num_of_prev_attempts'].astype('int8')
test['imd_band'] = test['imd_band'].astype('int16')
test['num_of_prev_attempts'] = test['num_of_prev_attempts'].astype('int8')
test['studied_credits'] = test['studied_credits'].astype('int16')
test['disability'] = test['disability'].astype('int8')
test['sum_click'] = test['sum_click'].astype('int32')
test['sum'] = test['sum'].astype('int8')
test['mean_date_submitted'] = test['mean_date_submitted'].astype('float64')
test['mean_weight'] = test['mean_weight'].astype('float64')
test['mean_date'] = test['mean_date'].astype('float64')

cols_to_convert = test.columns[10:]  # Выбираем столбцы с 10-го по последний
# Преобразуем типы данных столбцов с int64 на int8
test[cols_to_convert] = test[cols_to_convert].astype('int8')

# Переименование колонок

In [87]:
# Переименовываем колонки в датафрейме train_df
train_df.rename(columns={'region_East Midlands Region': 'region_East_Midlands_Region'}, inplace=True)
train_df.rename(columns={'region_London Region': 'region_London_Region'}, inplace=True)
train_df.rename(columns={'region_North Region': 'region_North_Region'}, inplace=True)
train_df.rename(columns={'region_North Western Region': 'region_North_Western_Region'}, inplace=True)
train_df.rename(columns={'region_South East Region': 'region_South_East_Region'}, inplace=True)
train_df.rename(columns={'region_South Region': 'region_South_Region'}, inplace=True)
train_df.rename(columns={'region_South West Region': 'region_South_West_Region'}, inplace=True)
train_df.rename(columns={'region_West Midlands Region': 'region_West_Midlands_Region'}, inplace=True)
train_df.rename(columns={'region_Yorkshire Region': 'region_Yorkshire_Region'}, inplace=True)
train_df.rename(columns={'highest_education_HE Qualification': 'highest_education_HE_Qualification'}, inplace=True)
train_df.rename(columns={'highest_education_Lower Than A Level': 'highest_education_Lower_Than_A_Level'}, inplace=True)
train_df.rename(columns={'highest_education_No Formal quals': 'highest_education_No_Formal_quals'}, inplace=True)
train_df.rename(columns={'highest_education_Post Graduate Qualification': 'highest_education_Post_Graduate_Qualification'}, inplace=True)
train_df.rename(columns={'age_band_35-55': 'age_band_35_55'}, inplace=True)
train_df.rename(columns={'age_band_55<=': 'age_band_55'}, inplace=True)

#####################################################
# Переименовываем колонки в датафрейме test
test.rename(columns={'region_East Midlands Region': 'region_East_Midlands_Region'}, inplace=True)
test.rename(columns={'region_London Region': 'region_London_Region'}, inplace=True)
test.rename(columns={'region_North Region': 'region_North_Region'}, inplace=True)
test.rename(columns={'region_North Western Region': 'region_North_Western_Region'}, inplace=True)
test.rename(columns={'region_South East Region': 'region_South_East_Region'}, inplace=True)
test.rename(columns={'region_South Region': 'region_South_Region'}, inplace=True)
test.rename(columns={'region_South West Region': 'region_South_West_Region'}, inplace=True)
test.rename(columns={'region_West Midlands Region': 'region_West_Midlands_Region'}, inplace=True)
test.rename(columns={'region_Yorkshire Region': 'region_Yorkshire_Region'}, inplace=True)
test.rename(columns={'highest_education_HE Qualification': 'highest_education_HE_Qualification'}, inplace=True)
test.rename(columns={'highest_education_Lower Than A Level': 'highest_education_Lower_Than_A_Level'}, inplace=True)
test.rename(columns={'highest_education_No Formal quals': 'highest_education_No_Formal_quals'}, inplace=True)
test.rename(columns={'highest_education_Post Graduate Qualification': 'highest_education_Post_Graduate_Qualification'}, inplace=True)
test.rename(columns={'age_band_35-55': 'age_band_35_55'}, inplace=True)
test.rename(columns={'age_band_55<=': 'age_band_55'}, inplace=True)

In [80]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26074 entries, 0 to 26073
Data columns (total 41 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   gender                                         26074 non-null  int8   
 1   imd_band                                       26074 non-null  int16  
 2   num_of_prev_attempts                           26074 non-null  int8   
 3   studied_credits                                26074 non-null  int16  
 4   disability                                     26074 non-null  int8   
 5   sum_click                                      26074 non-null  int32  
 6   mean_date_submitted                            26074 non-null  float64
 7   sum                                            26074 non-null  int8   
 8   mean_date                                      26074 non-null  float64
 9   mean_weight                                    26074 no

In [97]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split


# Разделение данных на обучающую и тестовую выборки
X_train = train_df.drop('weighted_score', axis=1)
y_train = train_df['weighted_score']
X_test = test

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [98]:
# Задаем набор параметров для RandomizedSearch
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.05, 0.1, 0.3],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

# Инициализируем модель XGBoost для регрессии
xgb_reg = xgb.XGBRegressor()

# Создаем объект RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_reg, param_distributions=param_grid, n_iter=100, cv=5, n_jobs=-1)

# Обучаем модель на обучающем наборе данных
random_search.fit(X_train, y_train)

# Предсказываем на тестовом наборе данных
y_pred = random_search.predict(X_test)

In [99]:
# Выводим лучшие параметры
print("Лучшие параметры:", random_search.best_params_)

# Выводим лучший результат
print("Лучший результат:", random_search.best_score_)

Лучшие параметры: {'subsample': 0.9, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
Лучший результат: 0.6031786715064935


In [94]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error

# # Оценка прогноза с использованием различных метрик
# mse = mean_squared_error(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)

In [112]:
# Сохранение прогнозова в файл
# Удаляем колонки 'code_module' и 'code_presentation'
Test_Who.drop(['code_module', 'code_presentation'], axis=1, inplace=True)

# Переименовываем колонку 'id_student' в 'weighted_score'
Test_Who.rename(columns={'id_student': 'weighted_score'}, inplace=True)
Test_Who['weighted_score'] = y_pred

Test_Who.to_csv('weighted_score.csv', index=False)